In [12]:
import os
import faiss
import numpy as np
import matplotlib.pyplot as plt
from tools.fileloader import load_xyz_rot
from mpl_toolkits import mplot3d

In [2]:
seqs = ["bomb_shelter", "botanical_garden", "bruin_plaza", "court_of_sciences", "dickson_court", "geo_loop",
        "kerckhoff", "luskin", "royce_hall", "sculpture_garden"]
seq = seqs[2]

root_folder = '/Volumes/vectr6/Dataset/overlap_transformer'
frames_poses_path = os.path.join(root_folder, 'poses', seq, 'poses.txt')
keyframes_poses_path = os.path.join(root_folder, 'keyframes', seq, 'poses', 'poses_kf.txt')
overlaps_path = os.path.join(root_folder, 'overlaps', f'{seq}.bin')


In [37]:
# load frames and keyframes positions
xyz, _ = load_xyz_rot(frames_poses_path)
xyz_kf, _ = load_xyz_rot(keyframes_poses_path)
overlaps = np.fromfile(overlaps_path)
overlaps = overlaps.reshape(-1, int(np.sqrt(len(overlaps))))

# add keyframes positions to faiss (search by L2 distance)
index = faiss.IndexFlatL2(3)
index.add(xyz_kf)
_, indices_euclidean = index.search(xyz, 5)

# search by overlaps
index_kf = faiss.IndexFlat(3)
index_kf.add(xyz)
_, indices_kf = index_kf.search(xyz_kf, 1)
overlaps_kf = overlaps[:, indices_kf.squeeze()]
indices_overlap = np.zeros((len(overlaps), 5), dtype=int)
for i in range(len(overlaps)):
    indices_i = overlaps_kf[i, :].argsort()[-5:][::-1]
    indices_overlap[i, :] = indices_i


In [38]:
%matplotlib qt
for i in range(xyz.shape[0]):
    xyz_curr = xyz[i, :]
    xyz_curr_kf = xyz_kf[indices_overlap[i, :], :]
    
    plt.clf()
    plt.scatter(xyz[:, 0], xyz[:, 1])
    plt.scatter(xyz_kf[:, 0], xyz_kf[:, 1], c='gold')
    plt.scatter(xyz_curr_kf[:, 0], xyz_curr_kf[:, 1], c='red')
    plt.scatter(xyz_curr[0], xyz_curr[1], c='violet')
    plt.show()
    plt.pause(0.01)
